In [1]:
import pandas as pd
import os
import pickle
import tqdm 
from utils import *

In [2]:
paths_mesh = read_paths_from_txt('/home/pelissier/These-ATER/Papier_international3/Dataset/paths_files/obj_files_ModelNet40_limper-projection-remeshing-752.txt')

dir_mesh = "/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_centered_scaled"
dir_rendu = "/home/pelissier/These-ATER/Papier_international3/Dataset/Rendu/ModelNet40/my_circular_12_elevation_30_remeshing"
dir_limper = "/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_limper_remeshing"

Pour 1 modèle 3D -- 12 pov

In [3]:
results = []
for path_mesh in tqdm.tqdm(paths_mesh):
    try :
    #if True:
        #path_mesh = "plant/train/plant_0053_SMPLER_centered_scaled_remeshing1"
        # Mesh courant
        #path_mesh = paths_mesh[0]; 
        #print(path_mesh)

        # Informations sur le mesh
        categorie, type = get_info_path(os.path.join(dir_mesh, path_mesh)); #print(categorie, type)
        name = os.path.basename(path_mesh); #print(name)
        # Saillance limper associée
        df_saillance_limper = pd.read_csv(os.path.join(dir_limper, path_mesh+"_limper_saliency_voronoi.csv"), header=None)
        saillance_limper = np.array(list((map(float, df_saillance_limper[6].tolist()[1:])))); #print(df_saillance_limper[6].tolist()[0])
        ## les valeurs des saillances sont déjà normalisées entre 0 et 1

        dict_scores = {}

        # Pour chaque pov
        for k in range(1, 13): 
            # data du pov k
            path_npz_cam_k = os.path.join(dir_rendu, path_mesh+"_cam"+str(k)+"_metadata_arrays.npz")
            # Load the .npz file
            data_cam_k = np.load(path_npz_cam_k)
            # sommets visibles
            sommets_visible = data_cam_k['vrai_idx_vert_visible']
            # Angles : cos(alpha)
            angles_visible = data_cam_k['arr_cos_vert'][np.where(data_cam_k['arr_cos_vert']>0)[0]]
            # Somme [limper*angle]
            saillance_limper_visible = saillance_limper[sommets_visible]
            terme_somme = np.sum(saillance_limper_visible*angles_visible)
            # Surface 3D + 'normalisation' : on divise par la surface 3D totale de l'objet
            terme_surface3d = data_cam_k['surface3D_visible']/data_cam_k['surface3D']
            # Sauvegardes des termes
            dict_scores['cam_'+str(k)] = {'terme_surface3d': terme_surface3d, 'saillance': terme_somme}

        # Normalisation du terme de saillance pour les 12 povs
        max_terme_saillance = np.max([dict_scores['cam_'+str(k)]['saillance'] for k in range(1, 13)])#; print(max_terme_saillance)
        dict_scores = {k: {'terme_surface3d': v['terme_surface3d'], 'saillance': v['saillance'], 
                            'terme_saillance': v['saillance']/max_terme_saillance} for k, v in dict_scores.items()}

        # Scores = (surface3D + saillance) pour les 12 povs
        for k, v in dict_scores.items():
            dict_scores[k]['score'] =  v['terme_surface3d'] + v['terme_saillance']

        # BVS 
        score_max = np.max([dict_scores[k]['score'] for k in dict_scores.keys()])
        dict_scores['bvs'] = [k for k in dict_scores.keys() if dict_scores[k]['score'] == score_max][0]
        # si plusieurs 
        if len([k for k in dict_scores.keys() if (('cam' in k) and (dict_scores[k]['score'] == score_max))]) > 1: print("Plusieurs pov", path_mesh)

        metadata = {
            "path_mesh": path_mesh, "categorie": categorie, "type": type, "name": name,
            "bvs" : dict_scores['bvs'], "score_max": score_max, "scores": dict_scores}

        with open(os.path.join(dir_rendu, path_mesh+"_bvs_bis.pkl"), "wb") as f: pickle.dump(metadata, f)
        
        # Ajout du résultat
        results.append(("ok", path_mesh))
    except:
        results.append(("pbl", path_mesh))
        

# Write results to file
with open("/home/pelissier/These-ATER/Papier_international3/Dataset/error_run2_bvs.txt", "w") as file:
    for name, path in results:
        file.write(f"{name}: {path}\n")

 22%|██▏       | 166/752 [00:04<00:12, 45.65it/s]/tmp/ipykernel_25611/12889981.py:14: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_saillance_limper = pd.read_csv(os.path.join(dir_limper, path_mesh+"_limper_saliency_voronoi.csv"), header=None)
 43%|████▎     | 322/752 [00:08<00:08, 49.46it/s]/tmp/ipykernel_25611/12889981.py:41: RuntimeWarning: invalid value encountered in scalar divide
  'terme_saillance': v['saillance']/max_terme_saillance} for k, v in dict_scores.items()}
100%|██████████| 752/752 [00:19<00:00, 39.50it/s]


In [5]:
dict_scores

{'cam_1': {'terme_surface3d': np.float64(0.9954244181871753),
  'saillance': np.float64(203.9280548290035),
  'terme_saillance': np.float64(0.7326477532795816),
  'score': np.float64(1.7280721714667568)},
 'cam_2': {'terme_surface3d': np.float64(0.9956931504084112),
  'saillance': np.float64(218.4892567048227),
  'terme_saillance': np.float64(0.7849614569939374),
  'score': np.float64(1.7806546074023486)},
 'cam_3': {'terme_surface3d': np.float64(0.9961980562167521),
  'saillance': np.float64(243.55440683573798),
  'terme_saillance': np.float64(0.8750124602481434),
  'score': np.float64(1.8712105164648953)},
 'cam_4': {'terme_surface3d': np.float64(0.9956558032490471),
  'saillance': np.float64(220.04059763723424),
  'terme_saillance': np.float64(0.790534924801763),
  'score': np.float64(1.7861907280508103)},
 'cam_5': {'terme_surface3d': np.float64(0.9959357368841939),
  'saillance': np.float64(226.26606149686825),
  'terme_saillance': np.float64(0.8129010093197006),
  'score': np.flo

Les erreurs peuvent venir des saillances qui sont a NAN ...